# Examples

This notebook allows to recreate the figures presented in the usage scenario section.

## Teaser

### Market Capitelization

Source: https://ycharts.com (Retrieved June 2, 2024)

In [34]:
import pandas as pd
market_cap = pd.read_parquet('https://paper.jupyter-scatter.dev/market-capitalization.pq')
market_cap.head(2)

,Symbol,Name,Sector,Industry,Date,Market Capitalization
0,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,1559260800000,947737317
1,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,1559520000000,918312097


In [39]:
import jscatter

scatter = jscatter.Scatter(data=market_cap, x='Date', x_scale='time', y='Market Capitalization')
scatter.axes(grid=True)
scatter.legend(True)
scatter.color(by='Sector')
scatter.connect(by='Symbol')
scatter.connection_color('inherit')
scatter.connection_size(3)
scatter.connection_opacity(0.5)
scatter.tooltip(True, preview='Name', properties=['x', 'y', 'color', 'Industry'])
scatter.width(640)
scatter.height(640)
scatter.show()

## GeoNames City Dataset

Source: https://geonames.org (Retrieved May 1, 2024)

In [1]:
import pandas as pd
geonames = pd.read_parquet('https://paper.jupyter-scatter.dev/geonames.pq')
geonames.head(2)

,Name,Longitude,Latitude,Mercator X,Mercator Y,Population,Continent,Country
0,Powidz,17.91926,52.41362,1994762.875,6875261.5,1000,Europe,Poland
1,Janowiec,21.88940,51.32359,2436716.750,6678734.0,1000,Europe,Poland


In [22]:
import jscatter

scatter = jscatter.Scatter(
    data=geonames,
    x='Longitude',
    y='Latitude',
    color_by='Continent',
    width=800,
    height=800
)
scatter.show()

In [23]:
from matplotlib.colors import AsinhNorm, LogNorm

scatter.opacity(0.5)
scatter.size(by='Population', map=(1, 8, 10), norm=AsinhNorm())
scatter.color(by='Population', map='magma', norm=LogNorm(), order='reverse')

In [24]:
scatter.legend(True)
scatter.axes(True, labels=True)
scatter.tooltip(True, properties=['color', 'Latitude', 'Country'], preview='Name')

In [26]:
scatter.widget.show_tooltip(114395)

In [ ]:
scatter.xy(x='Mercator X', y='Mercator Y')

In [25]:
scatter.selection(geonames.query('Population > 10_000_000').index)

In [ ]:
geonames.iloc[scatter.selection()]

## Fashion MNIST Embeddings

Source: Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms. Han Xiao, Kashif Rasul, Roland Vollgraf. [doi:10.48550/arXiv.1708.07747](https://doi.org/10.48550/arXiv.1708.07747)

In [27]:
import pandas as pd
fashion_mnist = pd.read_parquet('https://paper.jupyter-scatter.dev/fashion-mnist-embeddings.pq')

In [28]:
import anywidget
import traitlets
import traittypes

class ImagesWidget(anywidget.AnyWidget):
    _esm = """
    const baseUrl = 'https://paper.jupyter-scatter.dev/fashion-mnist-images/';
    export function render({ model, el }) {
      const container = document.createElement('div');
      container.classList.add('images-container');
      el.appendChild(container);

      const grid = document.createElement('div');
      grid.classList.add('images-grid');
      container.appendChild(grid);

      function renderImages() {
        grid.textContent = '';
        
        model.get('images').forEach((image) => {
          const imgId = String(image).padStart(5, '0');
        
          const img = document.createElement('div');
          img.classList.add('images-fashion-mnist');
          img.style.backgroundImage = `url(${baseUrl}${imgId}.png)`;
        
          grid.appendChild(img);
        });
      }

      model.on("change:images", renderImages);
      
      renderImages();
    }
    """

    _css = """
    .images-container {
      position: absolute;
      inset: 0;
      overflow: auto;
      background: black;
    }
    
    .images-grid {
      display: grid;
      grid-template-columns: repeat(auto-fit, minmax(32px, 1fr));
      align-content: flex-start;
      gap: 8px;
    }
    
    .images-fashion-mnist {
      width: 32px;
      height: 32px;
      background-repeat: no-repeat;
      background-position: center;
    }
    """

    images = traittypes.Array(default_value=[]).tag(sync=True)

In [31]:
import ipywidgets
import jscatter

images = ImagesWidget()

scatter = jscatter.Scatter(
    data=fashion_mnist,
    x='umapX',
    y='umapY',
    color_by='class',
    background_color='black',
    axes=False,
    width=800,
    height=800
)

ipywidgets.link((scatter.widget, 'selection'), (images, 'images'))

ipywidgets.AppLayout(center=scatter.show(), right_sidebar=images)

AppLayout(children=(ImagesWidget(layout=Layout(grid_area='right-sidebar')), HBox(children=(VBox(children=(Butt…

In [33]:
config = dict(
    data=fashion_mnist,
    background_color='black',
    color_by='class',
    legend=True,
    axes=False,
    zoom_on_selection=True,
)

pca = jscatter.Scatter(x='pcaX', y='pcaY', **config)
tsne = jscatter.Scatter(x='tsneX', y='tsneY', **config)
umap = jscatter.Scatter(x='umapX', y='umapY', **config)
cae = jscatter.Scatter(x='caeX', y='caeY', **config)

jscatter.compose(
    [(pca, "PCA"), (tsne, "t-SNE"), (umap, "UMAP"), (cae, "CAE")],
    sync_selection=True,
    sync_hover=True,
    rows=2,
    row_height=400,
)

GridBox(children=(VBox(children=(HTML(value='<b style="display: flex; justify-content: center; margin: 0 0 0 3…